In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq
)
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# Load required libraries and dataset
from datasets import load_dataset, Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq, TrainingArguments, Trainer


ds = load_dataset("lavita/MedQuAD")

def reduce_dataset(dataset, num_samples=5000, seed=42):
    return dataset.shuffle(seed=seed).select(range(num_samples))


dataset = reduce_dataset(ds['train'])

# Extract 'question' and 'answer' columns and convert to strings
questions = [str(item['question']) for item in dataset]
answers = [str(item['answer']) for item in dataset]


model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Create and preprocess dataset for training
def create_tokenized_dataset(questions, answers, tokenizer):
    data = {'question': questions, 'answer': answers}
    dataset = Dataset.from_dict(data)

    def preprocess_function(examples):
        inputs = [f"Question: {q}" for q in examples['question']]
        targets = examples['answer']
        model_inputs = tokenizer(inputs, max_length=512, truncation=True)
        labels = tokenizer(targets, max_length=512, truncation=True)
        model_inputs['labels'] = labels['input_ids']
        return model_inputs

    tokenized_dataset = dataset.map(preprocess_function, batched=True)
    return tokenized_dataset

# Tokenize the dataset
tokenized_dataset = create_tokenized_dataset(questions, answers, tokenizer)


data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=1000,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=500,
    evaluation_strategy="no",
)

# Set up the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset,
)

# Start training
trainer.train()

model.config.max_length = 512
model.config.length_penalty = 1.0
model.config.early_stopping = True

print("Training completed.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.77k [00:00<?, ?B/s]

(…)-00000-of-00001-e36383d177026d53.parquet:   0%|          | 0.00/10.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/47441 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

...

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,3.730600
1000,3.216300
1500,3.056900


Training completed.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Testing
def test_model(model, tokenizer, question):
  input_ids = tokenizer(f"Question: {question}", return_tensors="pt").input_ids.to(device)
  generated_ids = model.generate(input_ids=input_ids)
  answer = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return answer

test_questions = [
  "How many people are affected by Lowe syndrome ?"
]

for question in test_questions:
  answer = test_model(model, tokenizer, question)
  print(f"Question: {question}")
  print(f"Predicted Answer: {answer}\n")

Question: How many people are affected by Lowe syndrome ?
Predicted Answer: Lowe syndrome is a rare condition that affects a number of people. The condition is rare and may be caused by a number of people who have a low blood sugar. The majority of people with low blood sugar are treated in a uterine-like manner. The majority of people with low blood sugar are affected by lowe syndrome.



Llama 2-7b

In [ ]:

from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

In [ ]:

ds = load_dataset("lavita/MedQuAD")

def reduce_dataset(dataset, num_samples=5000, seed=42):
    return dataset.shuffle(seed=seed).select(range(num_samples))


dataset = reduce_dataset(ds['train'])

# Extract 'question' and 'answer' columns and convert to strings
questions = [str(item['question']) for item in dataset]
answers = [str(item['answer']) for item in dataset]


model_name = "NousResearch/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


def create_tokenized_dataset(questions, answers, tokenizer):
    data = {'question': questions, 'answer': answers}
    dataset = Dataset.from_dict(data)

    def preprocess_function(examples):
        inputs = [f"Question: {q}" for q in examples['question']]
        targets = examples['answer']
        model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding=True)
        labels = tokenizer(targets, max_length=512, truncation=True, padding=True)
        model_inputs['labels'] = labels['input_ids']
        return model_inputs

    tokenized_dataset = dataset.map(preprocess_function, batched=True)
    return tokenized_dataset


tokenized_dataset = create_tokenized_dataset(questions, answers, tokenizer)


data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=1000,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=500,
    evaluation_strategy="no",
)

# Set up the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset,
)

# Start training
trainer.train()

model.config.max_length = 512
model.config.length_penalty = 1.0
model.config.early_stopping = True

print("Training completed.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.77k [00:00<?, ?B/s]

(…)-00000-of-00001-e36383d177026d53.parquet:   0%|          | 0.00/10.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/47441 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]